In [50]:
from sklearn import datasets
import pandas as pd
import numpy as np
from keras.utils.np_utils import*
data = datasets.load_wine()
x_host=data.data[:,:8]
x_guest=data.data[:,8:]
feature_host=data.feature_names[:8]
feature_guest=data.feature_names[8:]
y=data.target
x_host=pd.DataFrame(x_host,columns=feature_host)
x_guest=pd.DataFrame(x_guest,columns=feature_guest)
y=to_categorical(y,num_classes=3)
y_true_pred=pd.DataFrame({"label":list(np.concatenate((y,np.zeros(np.shape(y))),axis=1))})
data_all = pd.concat([x_host, y_true_pred],axis=1)
build_tree_data=[data_all]
build_tree_node=["root"]
tree_dict={}
tree_branch_index=0
current_depth=0

In [51]:
n_estimators = 300
# 学习率
learning_rate = 0.001
# 结点分裂最小样本数
min_samples_split = 2
# 结点最小基尼不纯度
min_gini_impurity = 999
max_depth = 2
n_estimators = 300
# 学习率
learning_rate = 0.001
# 结点分裂最小样本数
min_samples_split =  2
# 结点最小基尼不纯度
min_gini_impurity = 999
# 树最大深度
max_depth =  2


In [52]:
# calucate_gini
class Sigmoid:
    def __call__(self, x):
        return 1 / (1 + np.exp(-x))

    def gradient(self, x):
        return self.__call__(x) * (1 - self.__call__(x))

# 定义Logit损失


class LogisticLoss:
    def __init__(self):
        sigmoid = Sigmoid()
        self._func = sigmoid
        self._grad = sigmoid.gradient

    # 定义损失函数形式
    def loss(self, y, y_pred):
        y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
        p = self._func(y_pred)
        return y * np.log(p) + (1 - y) * np.log(1 - p)

    # 定义一阶梯度
    def gradient(self, y, y_pred):
        p = self._func(y_pred)
        return -(y - p)

    # 定义二阶梯度
    def hess(self, y, y_pred):
        p = self._func(y_pred)
        return p * (1 - p)


def gain( y, y_pred, request="server"):
    loss=LogisticLoss()
    if request == "server":
        Gradient = np.power((y * loss.gradient(y, y_pred)).sum(), 2)
        # Hessian矩阵计算
        Hessian = loss.hess(y, y_pred).sum()
        return 0.5 * (Gradient / Hessian)
    elif request == "client":
        return y * loss.gradient(y, y_pred), loss.hess(y, y_pred)
def node_split( y):
    # 中间特征所在列
    feature = int(np.shape(y)[1]/2)
    # 左子树为真实值，右子树为预测值
    y_true, y_pred = y[:, :feature], y[:, feature:]
    return y_true, y_pred
def calculate_gini_impurity( y, y1, y2, request="server"):
    if request == "server":
        y = np.array(y.to_list())
        y1 = np.array(y1["label"].to_list())
        y2 = np.array(y2["label"].to_list())
        y_true, y_pred = node_split(y)
        y1, y1_pred = node_split(y1)
        y2, y2_pred = node_split(y2)
        true_gain = gain(y1, y1_pred, )
        false_gain = gain(y2, y2_pred, )
        gain_1 = gain(y_true, y_pred, )
        return true_gain + false_gain - gain_1
    else:
        y = np.array(y.to_list())
        y1 = np.array(y1["label"].to_list())
        y2 = np.array(y2["label"].to_list())
        y_true, y_pred = node_split(y)
        y1, y1_pred = node_split(y1)
        y2, y2_pred = node_split(y2)
        true_gain_gradient, true_gain_hess = gain(y1, y1_pred, request)
        false_gain_gradient, false_gain_hess = gain(y2, y2_pred, request)
        gain_gradient, gain_hess = gain(y_true, y_pred, request)
        return true_gain_gradient, true_gain_hess, false_gain_gradient, false_gain_hess, gain_gradient, gain_hess


def feature_split(X, feature_i, threshold):
    split_func = None
    if isinstance(threshold, int) or isinstance(threshold, float):
        def split_func(sample): return sample[feature_i] >= threshold
    else:
        def split_func(sample): return sample[feature_i] == threshold

    X_left = np.array([sample for sample in X if split_func(sample)])
    X_right = np.array([sample for sample in X if not split_func(sample)])
    return np.array([X_left, X_right])
data = build_tree_data.pop(0)
x=data.iloc[:,:-1]
y_true_pred=data.iloc[:,-1]
init_gini_impurity = 999
# 合并输入和标签
xy = pd.concat((x, y_true_pred), axis=1)
# 获取样本数和特征数
n_samples, n_features = x.shape
if n_samples >= min_samples_split and current_depth <= max_depth:
    # 遍历计算每个特征的基尼不纯度
    feature_list = feature_host
    for feature_i in range(n_features):
        # 获取第i特征的所有取值
        feature_values = np.expand_dims(x.iloc[:, feature_i], axis=1)
        # 获取第i个特征的唯一取值
        unique_values = np.unique(feature_values)
        # 遍历取值并寻找最佳特征分裂阈值
        for threshold in unique_values:
            # 特征节点二叉分裂
            xy1 = xy[xy[feature_list[feature_i]] >= threshold]
            xy2 = xy[xy[feature_list[feature_i]] < threshold]
            # xy1, xy2 = feature_split(xy, feature_i, threshold)
            # 如果分裂后的子集大小都不为0
            if len(xy1) > 0 and len(xy2) > 0:
                # 获取两个子集的标签值
                y1 = xy1.iloc[:, n_features:]
                y2 = xy2.iloc[:, n_features:]
                # 计算基尼不纯度
                # impurity = impurity_calculation(y, y1, y2)
                impurity = calculate_gini_impurity(y_true_pred, y1, y2)
                # 获取最小基尼不纯度
                # 最佳特征索引和分裂阈值
                if impurity < init_gini_impurity:
                    init_gini_impurity = impurity
                    best_criteria = {
                        "feature_name": feature_list[feature_i], "threshold": threshold}
                    best_sets = {
                        # "leftx": xy1.iloc[:, :n_features],
                        # "lefty": xy1.iloc[:, n_features:],
                        "left_index": xy1.index,
                        # "rightx": xy2.iloc[:, :n_features],
                        # "righty": xy2.iloc[:, n_features:],
                        "right_index": xy2.index,
                    }
    gain_server = init_gini_impurity
else:
    best_criteria = None
    best_sets = None
    gain_server = None
print(gain_server)

-3.552713678800501e-15


In [60]:
# encry_label
from phe import paillier
loss=LogisticLoss()
y_true, y_pred = node_split(np.array(y_true_pred.to_list()))
gradient_y = y_true * loss.gradient(y_true, y_pred)
hess_y=loss.hess(y_true, y_pred)
public_key, private_key = paillier.generate_paillier_keypair()
      # 加密整个数据
gradient_y_copy=[]
hess_y_copy=[]
for index, row in enumerate(gradient_y):
    gradient_y_copy.append([public_key.encrypt(x) for x in row])
for index, row in enumerate(hess_y):
    hess_y_copy.append([public_key.encrypt(x) for x in row])
gradient_y_copy = np.array(gradient_y_copy)
hess_y_copy = np.array(hess_y_copy)
gradient_y_copy = pd.DataFrame({"gradient":list(gradient_y_copy)}, index=y_true_pred.index)
hess_y_copy = pd.DataFrame({"hess":list(hess_y_copy)}, index=y_true_pred.index)



In [62]:
# guest  calu_encry_grdient_hess
def gradient_hess_sum(gradient_y,hess_y,gradient_y1,hess_y1,gradient_y2,hess_y2):
    gradient_y = np.array(gradient_y["gradient"].to_list())
    hess_y = np.array(hess_y["hess"].to_list())
    gradient_y1 = np.array(gradient_y1["gradient"].to_list())
    hess_y1 = np.array(hess_y1["hess"].to_list())
    gradient_y2 = np.array(gradient_y2["gradient"].to_list())
    hess_y2 = np.array(hess_y2["hess"].to_list())
    
    
    return gradient_y.sum(), hess_y.sum(), gradient_y1.sum(), hess_y1.sum(), gradient_y2.sum(), hess_y2.sum()
feature_splite={}
feature_splite_send = {}
     # 合并输入和标签
x_guest=x_guest.iloc[y_true_pred.index]
# xy = pd.concat((x_guest, gradient_y_copy,hess_y_copy), axis=1)
      # 获取样本数和特征数
n_samples, n_features = x_guest.shape
if n_samples >= min_samples_split and current_depth <= max_depth:
    # 遍历计算每个特征的基尼不纯度
    feature_list = x_guest.columns
    for feature_i in range(n_features):
        feature_splite[feature_i]=[]
        feature_splite_send[feature_i]=[]
        # 获取第i特征的所有取值
        feature_values = np.expand_dims(x_guest.iloc[:, feature_i], axis=1)
        # 获取第i个特征的唯一取值
        unique_values = np.unique(feature_values)

        # 遍历取值并寻找最佳特征分裂阈值
        for threshold in unique_values:
            # 特征节点二叉分裂
            x1 = x_guest[x_guest[feature_list[feature_i]] >= threshold]
            x2 = x_guest[x_guest[feature_list[feature_i]] < threshold]
            # xy1, xy2 = feature_split(xy, feature_i, threshold)
            # 如果分裂后的子集大小都不为0
            if len(x1) > 0 and len(x2) > 0:
                # 获取两个子集的标签值
                # y1 = xy1.iloc[:, n_features:]
                # y2 = xy2.iloc[:, n_features:]
                gradient_y1=gradient_y_copy.iloc[x1.index]
                gradient_y2 = gradient_y_copy.iloc[x2.index]
                hess_y1 = hess_y_copy.iloc[x1.index]
                hess_y2 = hess_y_copy.iloc[x2.index]
                # 计算基尼不纯度
                # impurity = self.impurity_calculation(y, y1, y2)
                feature_splite_send[feature_i].append({threshold: gradient_hess_sum(
                    gradient_y_copy, hess_y_copy, gradient_y1, hess_y1,gradient_y2,hess_y2)})
                best_criteria = {
                    "feature_i": feature_i, "threshold": threshold}
                best_sets = {
                    # "leftx": xy1[:, :n_features],
                    # "lefty": xy1[:, n_features:],
                    "left_index":x1.index,
                    "right_index":x2.index
                    # "rightx": xy2[:, :n_features],
                    # "righty": xy2[:, n_features:],
                    
                }
                feature_splite[feature_i].append({threshold:[best_criteria, best_sets]})
else:
    feature_splite_send = None
        # return {"host": feature_splite_send}


In [63]:
min_gain=999
for key in feature_splite_send:
    for value in feature_splite_send[key]:
        # feature_i=key
        # threshold=list(value.keys())[0]
        gain_value=list(value.values())[0]
        gain_real = 0.5 * \
            np.power(private_key.decrypt(
                gain_value[0]), 2)/private_key.decrypt(gain_value[1])
        gain_true = 0.5 * \
            np.power(private_key.decrypt(
                gain_value[2]), 2)/private_key.decrypt(gain_value[3])
        gain_false = 0.5 * \
            np.power(private_key.decrypt(
                gain_value[4]), 2)/private_key.decrypt(gain_value[5])
        gain_final=gain_true+gain_false-gain_real
        if gain_final<min_gain:
            min_gain=gain_final
            feature_i=key
            threshold=list(value.keys())[0]



-3.552713678800501e-15


In [ ]:
import pandas as pd
import numpy as np


class XGB:

    def __init__(self,
                 base_score=0.5,
                 max_depth=3,
                 n_estimators=10,
                 learning_rate=0.1,
                 reg_lambda=1,
                 gamma=0,
                 min_child_sample=None,
                 min_child_weight=1,
                 objective='linear'):

        self.base_score = base_score  # 最开始时给叶子节点权重所赋的值，默认0.5，迭代次数够多的话，结果对这个初值不敏感
        self.max_depth = max_depth  # 最大数深度
        self.n_estimators = n_estimators  # 树的个数
        self.learning_rate = learning_rate  # 学习率，别和梯度下降里的学习率搞混了，这里是每棵树要乘以的权重系数
        self.reg_lambda = reg_lambda  # L2正则项的权重系数
        self.gamma = gamma  # 正则项中，叶子节点数T的权重系数
        self.min_child_sample = min_child_sample  # 每个叶子节点的样本数（自己加的）
        self.min_child_weight = min_child_weight  # 每个叶子节点的Hessian矩阵和，下面代码会细讲
        self.objective = objective  # 目标函数，可选linear和logistic
        self.tree_structure = {}  # 用一个字典来存储每一颗树的树结构

    def xgb_cart_tree(self, X, w, m_dpth):
        '''
        递归的方式构造XGB中的Cart树
        X：训练数据集
        w：每个样本的权重值，递归赋值
        m_dpth：树的深度
        '''

        #边界条件：递归到指定最大深度后，跳出
        if m_dpth > self.max_depth:
            return

        best_var, best_cut = None, None
        #这里增益的初值一定要设置为0，相当于对树做剪枝，即如果算出的增益小于0则不做分裂
        max_gain = 0
        G_left_best, G_right_best, H_left_best, H_right_best = 0, 0, 0, 0
        #遍历每个变量的每个切点，寻找分裂增益gain最大的切点并记录下来
        for item in [x for x in X.columns if x not in ['g', 'h', 'y']]:
            for cut in list(set(X[item])):

                #这里如果指定了min_child_sample则限制分裂后叶子节点的样本数都不能小于指定值
                if self.min_child_sample:
                    if (X.loc[X[item] < cut].shape[0] < self.min_child_sample)\
                            | (X.loc[X[item] >= cut].shape[0] < self.min_child_sample):
                        continue

                G_left = X.loc[X[item] < cut, 'g'].sum()
                G_right = X.loc[X[item] >= cut, 'g'].sum()
                H_left = X.loc[X[item] < cut, 'h'].sum()
                H_right = X.loc[X[item] >= cut, 'h'].sum()

                #min_child_weight在这里起作用，指的是每个叶子节点上的H，即目标函数二阶导的加和
                #当目标函数为linear，即1/2*(y-y_hat)**2时，它的二阶导是1，那min_child_weight就等价于min_child_sample
                #当目标函数为logistic，其二阶导为sigmoid(y_hat)*(1-sigmoid(y_hat))，可理解为叶子节点的纯度，更详尽的解释可参看：
                #https://stats.stackexchange.com/questions/317073/explanation-of-min-child-weight-in-xgboost-algorithm#
                if self.min_child_weight:
                    if (H_left < self.min_child_weight) | (H_right < self.min_child_weight):
                        continue

                gain = G_left**2/(H_left + self.reg_lambda) + \
                    G_right**2/(H_right + self.reg_lambda) - \
                    (G_left + G_right)**2/(H_left + H_right + self.reg_lambda)
                gain = gain/2 - self.gamma
                if gain > max_gain:
                    best_var, best_cut = item, cut
                    max_gain = gain
                    G_left_best, G_right_best, H_left_best, H_right_best = G_left, G_right, H_left, H_right

        #如果遍历完找不到可分列的点，则返回None
        if best_var is None:
            return None

        #给每个叶子节点上的样本分别赋上相应的权重值
        id_left = X.loc[X[best_var] < best_cut].index.tolist()
        w_left = - G_left_best / (H_left_best + self.reg_lambda)

        id_right = X.loc[X[best_var] >= best_cut].index.tolist()
        w_right = - G_right_best / (H_right_best + self.reg_lambda)

        w[id_left] = w_left
        w[id_right] = w_right

        #用俄罗斯套娃式的json串把树的结构给存下来
        tree_structure = {(best_var, best_cut): {}}
        tree_structure[(best_var, best_cut)][('left', w_left)
                                             ] = self.xgb_cart_tree(X.loc[id_left], w, m_dpth+1)
        tree_structure[(best_var, best_cut)][('right', w_right)
                                             ] = self.xgb_cart_tree(X.loc[id_right], w, m_dpth+1)

        return tree_structure

    def _grad(self, y_hat, Y):
        '''
        计算目标函数的一阶导
        支持linear和logistic
        '''

        if self.objective == 'logistic':
            y_hat = 1.0/(1.0+np.exp(-y_hat))
            return y_hat - Y
        elif self.objective == 'linear':
            return y_hat - Y
        else:
            raise KeyError('objective must be linear or logistic!')

    def _hess(self, y_hat, Y):
        '''
        计算目标函数的二阶导
        支持linear和logistic
        '''

        if self.objective == 'logistic':
            y_hat = 1.0/(1.0+np.exp(-y_hat))
            return y_hat * (1.0 - y_hat)
        elif self.objective == 'linear':
            return np.array([1]*Y.shape[0])
        else:
            raise KeyError('objective must be linear or logistic!')

    def fit(self, X: pd.DataFrame, Y):
        '''
        根据训练数据集X和Y训练出树结构和权重
        '''

        if X.shape[0] != Y.shape[0]:
            raise ValueError('X and Y must have the same length!')

        X = X.reset_index(drop='True')
        Y = Y.values
        #这里根据base_score参数设定权重初始值
        y_hat = np.array([self.base_score]*Y.shape[0])
        for t in range(self.n_estimators):

            print('fitting tree {}...'.format(t+1))

            X['g'] = self._grad(y_hat, Y)
            X['h'] = self._hess(y_hat, Y)

            f_t = pd.Series([0]*Y.shape[0])
            self.tree_structure[t+1] = self.xgb_cart_tree(X, f_t, 1)

            y_hat = y_hat + self.learning_rate * f_t

            print('tree {} fit done!'.format(t+1))

        print(self.tree_structure)

    def _get_tree_node_w(self, X, tree, w):
        '''
        以递归的方法，把树结构解构出来，把权重值赋到w上面
        '''

        if not tree is None:
            k = list(tree.keys())[0]
            var, cut = k[0], k[1]
            X_left = X.loc[X[var] < cut]
            id_left = X_left.index.tolist()
            X_right = X.loc[X[var] >= cut]
            id_right = X_right.index.tolist()
            for kk in tree[k].keys():
                if kk[0] == 'left':
                    tree_left = tree[k][kk]
                    w[id_left] = kk[1]
                elif kk[0] == 'right':
                    tree_right = tree[k][kk]
                    w[id_right] = kk[1]

            self._get_tree_node_w(X_left, tree_left, w)
            self._get_tree_node_w(X_right, tree_right, w)

    def predict_raw(self, X: pd.DataFrame):
        '''
        根据训练结果预测
        返回原始预测值
        '''

        X = X.reset_index(drop='True')
        Y = pd.Series([self.base_score]*X.shape[0])

        for t in range(self.n_estimators):
            tree = self.tree_structure[t+1]
            y_t = pd.Series([0]*X.shape[0])
            self._get_tree_node_w(X, tree, y_t)
            Y = Y + self.learning_rate * y_t

        return Y

    def predict_prob(self, X: pd.DataFrame):
        '''
        当指定objective为logistic时，输出概率要做一个logistic转换
        '''

        Y = self.predict_raw(X)
        def sigmoid(x): return 1/(1+np.exp(-x))
        Y = Y.apply(sigmoid)

        return Y


In [6]:
feature_splite={0:[]}
best_criteria = {"feature_name": "a", "threshold": 0.8}
best_sets = {"leftx": 1,
            "lefty": 2,
            "rightx": 3,
            "righty":4,
                        }
feature_splite[0].append({0.8: [best_criteria, best_sets]})
for dict_threshold in feature_splite[0]:
    if list(dict_threshold.keys())[0] == 0.8:
        leftx = list(dict_threshold.values())[0][1]["leftx"]
        lefty = dict_threshold.values()[1]["lefty"]
        rightx = dict_threshold.values()[1]["rightx"]
        righty = dict_threshold.values()[1]["righty"]
        feature_name = dict_threshold.values()[0]["feature_name"]
        threshold = dict_threshold.values()[0]["threshold"]



TypeError: 'dict_values' object is not subscriptable